# Clase 3

#### Ejercicio:
Escribir un programa que reciba a través de la línea de comandos la cantidad de procesos hijos que debe crear. Cada proceso hijo debe dormir un numero aleatorio de segundos comprendido entre 0 y 30. El proceso padre debe esperar la terminación de cada uno de sus hijos. A medida que los hijos van terminando, el padre presentará en pantalla el pid de cada uno de ellos y la cantidad de segundos que ha estado durmiendo cada hijo. Hacerlo mediante la utilizacion de memoria compartida

In [1]:
#define _GNU_SOURCE

#include <stdio.h>
#include <unistd.h>
#include <sys/wait.h>
#include <stdlib.h>

/*#include <sys/types.h>*/
#include <sys/ipc.h>
#include <sys/shm.h>

#define NUMBER_OF_CHILDS 3
#define WAIT_TIME 10

long int rndom() {
    srand(getpid());
    long int result = rand() % WAIT_TIME;
    return result;
}

int main() {
    setbuf(stdout, NULL); //Disable stdout buffer

    int childs = NUMBER_OF_CHILDS;
    printf("Spawning %d process\n" , childs);
    
    key_t clave = ftok ( __FILE__ , 'a' );

   for (int i = 0; i < childs; i++) {
        int pid = fork();

        if (pid == 0) {
            int shmId = shmget ( clave,sizeof(int)*childs,0644|IPC_CREAT );
            int* tmpPtr = (int*) shmat ( shmId,NULL,0 );
            
            printf("Started process with id: %d\n" , getpid());
            unsigned int s = rndom();
            sleep(s);
            printf("Exited %d after -> %d\n",i,s);
            
            tmpPtr[i] = s;
            
            shmdt ( (void *) tmpPtr );
            
            exit(i);
        }

    }

    int shmId = shmget ( clave,sizeof(int)*childs,0644|IPC_CREAT );
    int* tmpPtr = (int*) shmat ( shmId,NULL,0 );
    
    for (int i = 0; i < childs; i++) {
        int a;
        wait(&a);
        printf("Kiled %d after %d\n",WEXITSTATUS(a),tmpPtr[WEXITSTATUS(a)]);
    }
    
    shmdt ( (void *) tmpPtr );
    shmctl ( shmId,IPC_RMID,NULL );
    
    return 0;
}

Spawning 3 process
Started process with id: 290
Started process with id: 291
Started process with id: 292
Exited 0 after -> 4
Kiled 0 after 4
Exited 1 after -> 5
Exited 2 after -> 5
Kiled 1 after 5
Kiled 2 after 5


In [1]:
ipcs


------ Message Queues --------
key        msqid      owner      perms      used-bytes   messages    

------ Shared Memory Segments --------
key        shmid      owner      perms      bytes      nattch     status      

------ Semaphore Arrays --------
key        semid      owner      perms      nsems     

